In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
df = pd.read_csv('datasets/aula03.csv')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,zipcode,lat,long,level,house_age,dormitory_type,condition_type,year,day,year_week
0,7129300520,2014-10-13,221900.00,3,1.00,1180,5650,1.00,0,0,...,98178,47.51,-122.26,low_level,new_house,house,regular,2014,2014-10-13,2014-41
1,6414100192,2014-12-09,538000.00,3,2.25,2570,7242,2.00,0,0,...,98125,47.72,-122.32,low_level,new_house,house,regular,2014,2014-12-09,2014-49
2,5631500400,2015-02-25,180000.00,2,1.00,770,10000,1.00,0,0,...,98028,47.74,-122.23,low_level,new_house,apartament,regular,2015,2015-02-25,2015-08
3,2487200875,2014-12-09,604000.00,4,3.00,1960,5000,1.00,0,0,...,98136,47.52,-122.39,high_level,new_house,house,good,2014,2014-12-09,2014-49
4,1954400510,2015-02-18,510000.00,3,2.00,1680,8080,1.00,0,0,...,98074,47.62,-122.05,low_level,new_house,house,regular,2015,2015-02-18,2015-07


# Respondendo as perguntas do CEO

In [3]:
# 1 - Qual a quantidade de imóveis por nível?
#   - Nível 0: Preço entre R$0.00 e R$321.950
#   - Nível 1: Preço entre R$321.950 e R$450.000
#   - Nível 2: Preço entre R$450.000 e R$645.000
#   - Nível 3: Preço acima de R$645.000

df.loc[(df.price > 0) & (df.price <= 321950), 'level'] = 0
df.loc[(df.price > 321950) & (df.price <= 450000), 'level'] = 1
df.loc[(df.price > 450000) & (df.price <= 645000), 'level'] = 2
df.loc[df.price > 645000, 'level'] = 3

df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,zipcode,lat,long,level,house_age,dormitory_type,condition_type,year,day,year_week
0,7129300520,2014-10-13,221900.00,3,1.00,1180,5650,1.00,0,0,...,98178,47.51,-122.26,0,new_house,house,regular,2014,2014-10-13,2014-41
1,6414100192,2014-12-09,538000.00,3,2.25,2570,7242,2.00,0,0,...,98125,47.72,-122.32,2,new_house,house,regular,2014,2014-12-09,2014-49
2,5631500400,2015-02-25,180000.00,2,1.00,770,10000,1.00,0,0,...,98028,47.74,-122.23,0,new_house,apartament,regular,2015,2015-02-25,2015-08
3,2487200875,2014-12-09,604000.00,4,3.00,1960,5000,1.00,0,0,...,98136,47.52,-122.39,2,new_house,house,good,2014,2014-12-09,2014-49
4,1954400510,2015-02-18,510000.00,3,2.00,1680,8080,1.00,0,0,...,98074,47.62,-122.05,2,new_house,house,regular,2015,2015-02-18,2015-07


In [ ]:
# 2 - Adicione as seguintes informações ao imóvel:
#   - Rua
#   - Número
#   - Bairro
#   - Cidade
#   - Estado

from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent='geoapiExercises')

df['house_number'] = 'NA'
df['road'] = 'NA'
df['neighbourhood'] = 'NA'
df['city'] = 'NA'
df['county'] = 'NA'
df['state'] = 'NA'

for i in range(len(df)):
    
    # make query 
    query = str(df.loc[i, 'lat']) + ',' + str(df.loc[i, 'long'])
    
    # API request
    response = geolocator.reverse(query)
    
    # populate data
    if 'house_number' in response.raw['address']:
        df.loc[i, 'house_number'] = response.raw['address']['house_number']
    if 'road' in response.raw['address']:
        df.loc[i, 'road'] = response.raw['address']['road']
    if 'neighbourhood' in response.raw['address']:
        df.loc[i, 'neighbourhood'] = response.raw['address']['neighbourhood']
    if 'city' in response.raw['address']:
        df.loc[i, 'city'] = response.raw['address']['city']
    if 'county' in response.raw['address']:
        df.loc[i, 'county'] = response.raw['address']['county']
    if 'state' in response.raw['address']:
        df.loc[i, 'state'] = response.raw['address']['state']

In [4]:
df.level = df.level.astype(int)

houses = df[['id', 'lat', 'long', 'price', 'level']].copy()

# Filtros iterativos no map

In [ ]:
import plotly.express as px

fig = px.scatter_mapbox( houses,
                         lat = 'lat',
                         lon = 'long',
                         color = 'level',
                         size = 'price',
                         color_continuous_scale = px.colors.cyclical.IceFire,
                         size_max = 15,
                         zoom = 10 )

fig.update_layout(mapbox_style='open-street-map')
fig.update_layout(height=600, margin={'r': 0, 't': 0, 'l': 0, 'r': 0})
fig.show()

In [ ]:
import ipywidgets as widgets

df['is_waterfront'] = df['waterfront'].apply(lambda x: 'yes' if x == 1 else 'no')

df['level'] = df['price'].apply(lambda x: 0 if x < 321950 else
                                          1 if (x > 321950) & (x <= 450000) else
                                          2 if (x > 450000) & (x <= 645000) else 3)

df['level'] = df['level'].astype(int)
style = {'description_width': 'initial'}

#interative buttons
price_limit = widgets.IntSlider(
    value = 540000,
    min = 75000,
    max = 77000000,
    step = 1,
    description = 'Maximun Price',
    disable = False,
    style = style
)

waterfront_bar = widgets.Dropdown(
    options = df.is_waterfront.unique().tolist(),
    value='yes',
    description='Water View',
    disable = False
)

def update_map(df, waterfront, limit):
    houses = df[(df.price <= limit) & (df.is_waterfront == waterfront)][['id', 'lat', 'long', 'price', 'level']].copy()
    
    fig = px.scatter_mapbox( houses,
                         lat = 'lat',
                         lon = 'long',
                         color = 'level',
                         size = 'price',
                         color_continuous_scale = px.colors.cyclical.IceFire,
                         size_max = 15,
                         zoom = 10 )

    fig.update_layout(mapbox_style='open-street-map')
    fig.update_layout(height=600, margin={'r': 0, 't': 0, 'l': 0, 'r': 0})
    fig.show()

In [ ]:
from ipywidgets import fixed

widgets.interactive(update_map, df=fixed(df), waterfront=waterfront_bar, limit=price_limit)

# Dashboard Interativo

In [28]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib import gridspec

In [29]:
df.drop(['year', 'day', 'year_week'], axis=1)

# change date format
df['year'] = pd.to_datetime(df.date).dt.strftime('%Y')
df['date'] = pd.to_datetime(df.date).dt.strftime('%Y-%m-%d')
df['year_week'] = pd.to_datetime(df.date).dt.strftime('%Y-%U')

# Widgets to control data
date_limit = widgets.SelectionSlider(
    options = df.date.sort_values().unique().tolist(),
    value = '2014-12-01',
    description = 'Disponivel',
    continuous_update = False,
    orientation = 'horizontal',
    readout = True
)

def update_map(df, limit):
    # Filter data
    df = df[df.date >= limit].copy()
    
    fig = plt.figure(figsize=(21,12))
    specs = gridspec.GridSpec(ncols = 2, nrows=2, figure=fig)
    
    ax1 = fig.add_subplot(specs[0, :])
    ax2 = fig.add_subplot(specs[1, 0])
    ax3 = fig.add_subplot(specs[1, 1])
    
    by_year = df[['id', 'year']].groupby('year').sum().reset_index()
    ax1.bar(by_year.year, by_year.id)
    
    by_day = df[['id', 'date']].groupby('date').mean().reset_index()
    ax2.plot(by_day.date, by_day.id)
    ax2.set_title('Avg price by day')
    
    by_week = df[['id', 'year_week']].groupby('year_week').mean().reset_index()
    ax3.plot(by_week.year_week, by_week.id)
    ax3.set_title('Avg price by week of year')
    plt.xticks(rotation=60);
    

In [30]:
from ipywidgets import fixed

widgets.interactive(update_map, df=fixed(df), limit=date_limit)

interactive(children=(SelectionSlider(continuous_update=False, description='Disponivel', index=212, options=('…

# Novas perguntas do CEO

In [5]:
# 1 - Qual a média dos preços dos imóveis por nível?
#   - Nível 0: Preço entre R$0.00 e R$321.950
#   - Nível 1: Preço entre R$321.950 e R$450.000
#   - Nível 2: Preço entre R$450.000 e R$645.000
#   - Nível 3: Preço acima de R$645.000

df['level'] = df['price'].apply(lambda x: 0 if x <= 321950 else
                               1 if (x > 321950) & (x <= 450000 ) else
                               2 if (x > 450000) & (x <= 645000 ) else 3)

df[['price', 'level']].groupby('level').mean().reset_index()

,level,price
0,0,251557.65
1,1,385688.68
2,2,539730.96
3,3,987540.22


In [6]:
# 2 - Qual a média do tamanho da sala de estar por "Size"?
#   - Size 0: entre 0 e 1427 sqft
#   - Size 1: entre 1427 e 1910 sqft
#   - Size 2: entre 1910 e 2550
#   - Size 3: acima de 2550 sqft

df['size'] = df['sqft_living'].apply(lambda x: 0 if x <= 1427 else
                                    1 if (x > 1427) & (x <= 1910) else
                                    2 if (x > 1910) & (x <= 2550) else 3)

df[['size', 'sqft_living']].groupby('size').mean().reset_index()

,size,sqft_living
0,0,1123.83
1,1,1664.96
2,2,2211.79
3,3,3329.61


In [ ]:
# 3 - Adicione as seguintes informações ao conjunto de dados original:
# - Place ID: Identificação da localização
# - OSM Type: Open Street Map Type
# - Country: Nome do País
# Country Code: Código do País

from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent='geoapiExercises')

df['place_id'] = 'NA'
df['osm_type'] = 'NA'
df['country'] = 'NA'
df['country_code'] = 'NA'

for i in range(len(df)):
    
    # make query 
    query = str(df.loc[i, 'lat']) + ',' + str(df.loc[i, 'long'])
    
    # API request
    response = geolocator.reverse(query)
    
    # populate data
    if 'place_id' in response.raw:
        df.loc[i, 'place_id'] = response.raw['place_id']
    if 'osm_type' in response.raw:
        df.loc[i, 'osm_type'] = response.raw['osm_type']
    if 'country' in response.raw['address']:
        df.loc[i, 'country'] = response.raw['address']['country']
    if 'country_code' in response.raw['address']:
        df.loc[i, 'country_code'] = response.raw['address']['country_code']

In [40]:
# 4 - Adicione os seguintes filtros no Mapa:
# - Tamanho mínimo da área da sala de estar 
# - Número mínimo de banheiros
# - Valor máximo de preço 
# - Tamanho máximo da área do porão
# - Filtro das condições do imóvel
# - Filtro por ano de construção

df.level = df.level.astype(int)

import plotly.express as px
import ipywidgets as widgets
from ipywidgets import fixed

style = {'description_width': 'initial'}

#interative buttons

living_limit = widgets.IntSlider(
    value = 1910.00,
    min = 290.00,
    max =  13540.00,
    step = 1,
    description = 'Minimun Living Room',
    disable = False,
    style = style
)

bathroom_limit = widgets.IntSlider(
    value = 1,
    min = 0,
    max =  8,
    step = 1,
    description = 'Minimun Bathrooms',
    disable = False,
    style = style
)

price_limit = widgets.IntSlider(
    value = 540000,
    min = 75000,
    max = 77000000,
    step = 1,
    description = 'Maximun Price',
    disable = False,
    style = style
)

basement_limit = widgets.IntSlider(
    value = 0,
    min = 0,
    max =  4820.00,
    step = 1,
    description = 'Maximun Basement Size',
    disable = False,
    style = style
)

condition_bar = widgets.Dropdown(
    options = df.condition_type.unique().tolist(),
    value='regular',
    description='Condition Type',
    disable = False
)

yr_built_bar = widgets.Dropdown(
    options = df.yr_built.sort_values().unique().tolist(),
    value='2000-01-01',
    description='Year Built',
    disable = False
)


def update_map(df, living_limit, bathroom_limit, price_limit, basement_limit, condition_bar, yr_built_bar):
    houses = df[(df.price <= price_limit) & (df.sqft_living >= living_limit) & (df.bathrooms >= bathroom_limit)
               & (df.sqft_basement <= basement_limit) & (df.condition_type == condition_bar) & (df.yr_built <= yr_built_bar)]
    [['id', 'lat', 'long', 'price', 'level', 'sqft_living', 'bathrooms', 'sqft_basement', 'yr_built', 'condition_type']].copy()
    
    fig = px.scatter_mapbox( houses,
                         lat = 'lat',
                         lon = 'long',
                         color = 'level',
                         size = 'price',
                         color_continuous_scale = px.colors.cyclical.IceFire,
                         size_max = 15,
                         zoom = 10 )

    fig.update_layout(mapbox_style='open-street-map')
    fig.update_layout(height=600, margin={'r': 0, 't': 0, 'l': 0, 'r': 0})
    fig.show()

In [41]:
from ipywidgets import fixed

widgets.interactive(update_map, df=fixed(df), living_limit=living_limit, price_limit=price_limit, bathroom_limit=bathroom_limit,
                   basement_limit=basement_limit, condition_bar=condition_bar, yr_built_bar=yr_built_bar)

interactive(children=(IntSlider(value=1910, description='Minimun Living Room', max=13540, min=290, style=Slide…

In [ ]:
# 5 - Adicione os seguintes filtros no Dashboard:
# - Filtro por data disponível para compra
# - Filtro por ano de renovação
# - Filtro se possui vista para a água

In [37]:
df['is_waterfront'] = df['waterfront'].apply(lambda x: 'yes' if x == 1 else 'no')

# Widgets to control data
date_available = widgets.SelectionSlider(
    options = df.date.sort_values().unique().tolist(),
    value = '2014-12-01',
    description = 'Available',
    continuous_update = False,
    orientation = 'horizontal',
    readout = True
)

yr_renovated = widgets.SelectionSlider(
    options = df.yr_renovated.sort_values().unique().tolist(),
    value = 2002,
    description = 'Year Renovated',
    continuous_update = False,
    orientation = 'horizontal',
    readout = True
)

waterfront_bar = widgets.Dropdown(
    options = df.is_waterfront.unique().tolist(),
    value='yes',
    description='Water View',
    disable = False
)

def update_map(df, date_available, yr_renovated, waterfront):
    # Filter data
    df = df[(df.date >= date_available) & (df.yr_renovated == yr_renovated) & (df.is_waterfront == waterfront)].copy()
    
    fig = plt.figure(figsize=(21,12))
    specs = gridspec.GridSpec(ncols = 2, nrows=2, figure=fig)
    
    ax1 = fig.add_subplot(specs[0, :])
    ax2 = fig.add_subplot(specs[1, 0])
    ax3 = fig.add_subplot(specs[1, 1])
    
    by_year = df[['id', 'year']].groupby('year').sum().reset_index()
    ax1.bar(by_year.year, by_year.id)
    
    by_day = df[['id', 'date']].groupby('date').mean().reset_index()
    ax2.plot(by_day.date, by_day.id)
    ax2.set_title('Avg price by day')
    
    by_week = df[['id', 'year_week']].groupby('year_week').mean().reset_index()
    ax3.plot(by_week.year_week, by_week.id)
    ax3.set_title('Avg price by week of year')
    plt.xticks(rotation=60);
    

In [39]:
from ipywidgets import fixed

widgets.interactive(update_map, df=fixed(df), date_available=date_available, yr_renovated=yr_renovated, 
                    waterfront=waterfront_bar)

interactive(children=(SelectionSlider(continuous_update=False, description='Available', index=212, options=('2…

In [4]:
df.to_csv('datasets/aula04.csv')